In [1]:
from utils.sampling import *
from utils.config import cfg_from_yaml_file
from tools import builder
from datasets.data_transforms import ResamplePoints, LidarSimulation
from datasets import build_dataset_from_cfg
import open3d as o3d
import numpy as np
import glob
from utils.vis_utils import convert_to_o3dpcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
from pathlib import Path
import pickle

data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/val')
modelid_path = data_dir / 'model_ids.txt'

with open(modelid_path, 'r') as f:
    model_ids = [line.split('\n')[0] for line in f.readlines()]

partial_points_path = str(data_dir / 'partial' / '%s' / '%03d.pcd')
surface_points_path = str(data_dir / 'surface' / '%s' / '%03d.pcd')
label_path = str(data_dir / 'label.pkl')

with open(label_path, 'rb') as f:
    label = pickle.load(f)

In [99]:
from tqdm import tqdm 

lidarsim = LidarSimulation({})

dataset_size = 2000
data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test')
partial_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/partial')
surface_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/surface')
num_exported = 0

new_labels = {}
for i in tqdm(range(len(model_ids)), total=len(model_ids)):
    
    m_id = model_ids[i]  
    new_labels[m_id] = {}
    
    view_ids = np.random.default_rng().choice(20, size=6, replace=False)
    for v_id in view_ids:             
        partial_path = partial_points_path % (m_id, v_id)
        surface_path = surface_points_path % (m_id, v_id)

        pcd = o3d.io.read_point_cloud(partial_path)
        pc_np = np.asarray(pcd.points)
        pc_np_sim = lidarsim(pc_np)
        partial_pc_path = partial_dir / m_id
        partial_pc_path.mkdir(exist_ok=True, parents=True)        
        o3d.io.write_point_cloud(str(partial_pc_path / f'{v_id:03d}.pcd'), convert_to_o3dpcd(pc_np_sim))
        
        surface_pcd = o3d.io.read_point_cloud(surface_path)
        surface_pc_path = surface_dir / m_id
        surface_pc_path.mkdir(exist_ok=True, parents=True) 
        o3d.io.write_point_cloud(str(surface_pc_path / f'{v_id:03d}.pcd'), surface_pcd)
        
        new_labels[m_id][v_id] = label[m_id][v_id]
        new_labels[m_id][v_id]['num_pts'] = pc_np_sim.shape[0]        
        num_exported += 1
        
        if num_exported >= dataset_size:
            break
            
    if num_exported >= dataset_size:
        break
        
with open(str(data_dir / 'label.pkl'), 'wb') as f:
    pickle.dump(new_labels, f)

 98%|████████████████████████████████████████████ | 333/340 [00:23<00:00, 13.97it/s]


In [3]:
pth = '/VCN/data/shapenet/VC/vc-surface_nviews-20/test/partial/1a3782ae4bd711b66b418c7d9fedcaa9/008.pcd'

pcd = o3d.io.read_point_cloud(pth)

In [4]:
o3d.visualization.draw_geometries([pcd])